In [1]:
%env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [2]:
#讀入相關套件
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils

In [3]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [4]:
#顯示資料長度
print('總共有 %d 訓練資料，每筆資料的尺寸 %d * %d'%x_train.shape)
print('總共有 %d 測試資料，每筆資料的尺寸 %d * %d'%x_test.shape)

總共有 60000 訓練資料，每筆資料的尺寸 28 * 28
總共有 10000 測試資料，每筆資料的尺寸 28 * 28


In [9]:
#輸入格式整理
x_train=x_train.reshape(60000,28,28,1)
x_test=x_test.reshape(10000,28,28,1)

In [10]:
#將訓練與測試資料中數字為 0和 1的資料篩選出來
x_train_01=x_train[y_train<=1]
x_test_01=x_test[y_test<=1]

In [11]:
#將標籤轉換成one-hot encoding形式
y_train_10=np_utils.to_categorical(y_train,10)
y_test_10=np_utils.to_categorical(y_test,10)

y_train_01=y_train[y_train<=1]
y_test_01=y_test[y_test<=1]

y_train_01=np_utils.to_categorical(y_train_01,2)
y_test_01=np_utils.to_categorical(y_test_01,2)

In [12]:
#檢視資料大小
x_train_01.shape,y_train_01.shape

((12665, 28, 28, 1), (12665, 2))

In [13]:
x_test_01.shape,y_test_01.shape

((2115, 28, 28, 1), (2115, 2))

In [14]:
#建構神經網路
model=Sequential()
model.add(Conv2D(32,(3,3),padding='same',input_shape=(28,28,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 128)       

In [15]:
model.layers

In [16]:
#以list形式使用Sequential API
conv_layer=[Conv2D(32,(3,3),padding='same',input_shape=(28,28,1)),
           Activation('relu'),
           MaxPooling2D(pool_size=(2,2)),
           
           Conv2D(64,(3,3),padding='same'),
           Activation('relu'),
           MaxPooling2D(pool_size=(2,2)),
           
           Conv2D(128,(3,3),padding='same'),
           Activation('relu'),
           MaxPooling2D(pool_size=(2,2))]

fc_layer=[Flatten(),
         Dense(200),Activation('relu'),
         Dense(10),Activation('softmax')]
model=Sequential(conv_layer+fc_layer)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_6 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_7 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 128)        

In [17]:
model.load_weights('handwriting_weights_cnn.h5')

In [18]:
#保留前三層convolutional layer，並進行轉移學習
new_fc_layer=[Flatten(),
         Dense(200),Activation('relu'),
         Dense(2),Activation('softmax')]
model_0_to_1=Sequential(conv_layer+new_fc_layer)
model_0_to_1.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_6 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_7 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 128)        

In [19]:
for layer in conv_layer:
    layer.trainable=False

In [20]:
#檢視進行轉移學習後的神經網路結構
model_0_to_1.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_6 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_7 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 128)        

In [24]:
model_0_to_1.compile(loss='mse',optimizer=SGD(lr=0.05),metrics=['accuracy'])

In [25]:
#訓練神經網路
model_0_to_1.fit(x_train_01,y_train_01,batch_size=100,epochs=5)


Epoch 1/5
12665/12665 [==============================] - 8s 593us/step - loss: 0.5305 - accuracy: 0.4689
Epoch 2/5
12665/12665 [==============================] - 7s 545us/step - loss: 0.0149 - accuracy: 0.9843
Epoch 3/5
12665/12665 [==============================] - 7s 550us/step - loss: 0.0021 - accuracy: 0.9975
Epoch 4/5
12665/12665 [==============================] - 8s 598us/step - loss: 8.5652e-04 - accuracy: 0.9989
Epoch 5/5
12665/12665 [==============================] - 7s 532us/step - loss: 5.9058e-04 - accuracy: 0.9994


In [27]:
#顯示損失分數與準確度
score=model_0_to_1.evaluate(x_test_01,y_test_01)

print('測試資料的loss:',score[0])
print('測試資料正確率:',score[1])

測試資料的loss: 0.00024808209705341744
測試資料正確率: 0.9995272159576416
